In [12]:
from pettingzoo.mpe import simple_adversary_v3

env = simple_adversary_v3.parallel_env(render_mode="human")
observations, infos = env.reset()
print('observations:',observations)
print('infos:',infos)
print('agents:',env.agents)
action = {agent: env.action_space(agent).sample() for agent in env.agents}
print('action:',action)
observations, rewards, terminations, truncations, infos = env.step(action)
print('observations:',observations)
print('rewards:',rewards)
print('terminations:',terminations)
print('truncations:',truncations)
print('infos:',infos)
env.close()
print('state_dim:',env.observation_space('agent_0').shape[0])
print('action_dim:',env.action_space('agent_0').n)   #env.action_spaces['agent_0'].n) 是个字典
print('max_num_agents:',env.max_num_agents)
print('n_agents:',env.num_agents)

observations: {'adversary_0': array([-0.32956174, -1.108241  , -0.7522789 ,  0.12979814, -0.35218278,
       -1.3779373 , -0.8661719 , -1.4783411 ], dtype=float32), 'agent_0': array([-0.40009615,  1.5077355 ,  0.02262103,  0.26969635, -0.40009615,
        1.5077355 ,  0.35218278,  1.3779373 , -0.51398915, -0.10040382],
      dtype=float32), 'agent_1': array([0.11389301, 1.6081393 , 0.5366102 , 0.37010017, 0.11389301,
       1.6081393 , 0.8661719 , 1.4783411 , 0.51398915, 0.10040382],
      dtype=float32)}
infos: {'adversary_0': {}, 'agent_0': {}, 'agent_1': {}}
agents: ['adversary_0', 'agent_0', 'agent_1']
action: {'adversary_0': 4, 'agent_0': 2, 'agent_1': 1}
observations: {'adversary_0': array([-0.32956174, -1.108241  , -0.7522789 ,  0.12979814, -0.35218278,
       -1.3779373 , -0.8661719 , -1.4783411 ], dtype=float32), 'agent_0': array([-0.40009615,  1.5077355 ,  0.02262103,  0.26969635, -0.40009615,
        1.5077355 ,  0.35218278,  1.3779373 , -0.51398915, -0.10040382],
      dtyp

In [20]:
from MADDPG import MADDPG
import torch
from pettingzoo.mpe import simple_adversary_v3, simple_spread_v3, simple_tag_v3
def get_env(env_name, ep_len=25):
    """create environment and get observation and action dimension of each agent in this environment"""
    new_env = None
    if env_name == 'simple_adversary_v3':
        new_env = simple_adversary_v3.parallel_env(max_cycles=ep_len,render_mode='rgb_array')
    if env_name == 'simple_spread_v3':
        new_env = simple_spread_v3.parallel_env(max_cycles=ep_len)
    if env_name == 'simple_tag_v3':
        new_env = simple_tag_v3.parallel_env(max_cycles=ep_len)

    new_env.reset()
    _dim_info = {}
    for agent_id in new_env.agents:
        _dim_info[agent_id] = []  # [obs_dim, act_dim]
        _dim_info[agent_id].append(new_env.observation_space(agent_id).shape[0])
        _dim_info[agent_id].append(new_env.action_space(agent_id).n)

    return new_env, _dim_info
dim_info = get_env('simple_adversary_v3')[1]
#dim_info, capacity, batch_size, actor_lr, critic_lr, res_dir)
maddpg = MADDPG(dim_info,1,36,0.01,0.01,'test')

print(maddpg.select_action(observations))

{'adversary_0': 1, 'agent_0': 0, 'agent_1': 1}


In [1]:
from collections import defaultdict

# 创建一个 defaultdict，默认值类型为 int
dd = defaultdict(int)

# 向 defaultdict 中添加键值对
dd['adversary_0'] = -0.2605955550976603
dd['agent_0'] = -0.8693780243973747
dd['agent_1'] = -0.8693780243973747

# 打印 defaultdict
print(dd)


defaultdict(<class 'int'>, {'adversary_0': -0.2605955550976603, 'agent_0': -0.8693780243973747, 'agent_1': -0.8693780243973747})


In [9]:
from Agent import Agent
import torch

agent = Agent(12,1,36,0.01,0.01)
obs = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12],dtype=torch.float32)
print(agent.action(obs))
print(agent.target_action(obs))


tensor([1.], grad_fn=<AddBackward0>)
tensor(1.)


In [4]:
import math
print(math.cos(0))
print(math.cos(math.pi/2))
print(math.cos(math.pi))
print(math.cos(3*math.pi/2))
print(math.cos(2*math.pi))

print(math.tanh(0))

1.0
6.123233995736766e-17
-1.0
-1.8369701987210297e-16
1.0
0.0007963267107332633


In [8]:
import numpy as np
action =1
action = action + 0.1* np.random.randn() # np.random.randn(1) 1有增加array维度的作用    
print(action)


[1.05185142]


array([-0.34829525, -1.38448912])

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128,):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)
        '''
        #根据计算增益
        gain1 = nn.init.calculate_gain('relu')
        gain2 = nn.init.calculate_gain('tanh')
        #Xavier均匀分布初始化
        torch.nn.init.xavier_uniform_(self.fc1.weight, gain=gain1)
        torch.nn.init.xavier_uniform_(self.fc2.weight, gain=gain2)
        #初始化参数
        #self.fc1.weight.data.normal_(0, 0.1)
        self.fc1.bias.data.fill_(0.01)
        #self.fc2.weight.data.normal_(0, 0.1)
        self.fc2.bias.data.fill_(0.01)

        #self.action_bound = action_bound  # action_bound是环境可以接受的动作最大值
        '''
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.tanh(self.fc2(x)) # tanh输出范围是(-1,1)，符合动作的范围 还需要后续clip
policy = PolicyNet(12,1)
obs = torch.tensor([1,2,3,4,5,6,7,8,9,10,11,12],dtype=torch.float32)
print(policy(obs))
torch.randn(1) 

tensor([-0.7499], grad_fn=<TanhBackward0>)


tensor([-0.0958])

In [73]:
import numpy as np
win_list = [1,0,0,0,0,0,0,0,0,0]
win_array = np.mean(np.array(win_list).reshape(-1, 10), axis=1) #axis=1表示按行求均值
print(win_array)
print(np.array(win_list).reshape(-1, 10))

[0.1]
[[1 0 0 0 0 0 0 0 0 0]]


In [2]:
done ={'Red-0': False, 'Red-1': False, 'Red-2': False}
print(any(done.values()))

False


In [3]:
import pickle

# 假设你有一个名为 'data.pkl' 的文件
with open('results\\toy_env\\24\\rewards.pkl', 'rb') as file:
    data = pickle.load(file)

# 现在 'data' 变量包含了从 .pkl 文件中恢复的对象
print(data)


{'rewards': {'Red-0': array([ -4.        ,  -6.64236263,  -6.39353434, ..., 183.4901285 ,
       184.23064647, 188.91      ]), 'Red-1': array([128.31      ,  -5.24634945,  -6.43049854, ..., 185.66645085,
       194.87857453, 166.50867731]), 'Red-2': array([ -4.        ,  -4.02393492,  -8.36051615, ..., 187.9       ,
        14.18      , 187.9       ])}}


In [1]:
angle=[3.00596919]
assert 0 <= angle <= 2 * math.pi
print(angle)

TypeError: '<=' not supported between instances of 'int' and 'list'

In [7]:
import numpy as np
red0_hp_l = [0]
print(np.mean(red0_hp_l))

0.0


In [1]:
1e-3

0.001

In [1]:
4 // 4


1

In [2]:
from myMADDPG import MADDPG
from toy_env import ToyEnv

def get_env(env_name):
    """create environment and get observation and action dimension of each agent in this environment"""
    new_env = None
    if env_name == 'toy_env':
        new_env = ToyEnv()


    new_env.reset()
    _dim_info = {}
    for agent_id in new_env.agents:
        _dim_info[agent_id] = []  # [obs_dim, act_dim]
        _dim_info[agent_id].append(new_env.observation_space(agent_id).shape[0])
        _dim_info[agent_id].append(new_env.action_space(agent_id).shape[0])

    return new_env, _dim_info
dim_info = get_env('toy_env')[1]


global_obs_act_dim = sum(sum(val) for val in dim_info.values())
print(global_obs_act_dim) #13 *3

39


In [3]:
import torch
import torch.nn as nn

# 假设 batch_size = 2, attention_dim = 5
batch_size = 2
attention_dim = 5

# 创建一个形状为 (2, 1, 13) 的输入张量
input_tensor = torch.randn(batch_size, 1, 13)

# 定义线性层
embedding = nn.Linear(13, attention_dim)

# 通过线性层
output_tensor = embedding(input_tensor)

print("输入张量形状:", input_tensor.shape)
print("输出张量形状:", output_tensor.shape)


输入张量形状: torch.Size([2, 1, 13])
输出张量形状: torch.Size([2, 1, 5])


In [5]:
import torch

# 创建一个形状为 (2, 3) 的张量
tensor = torch.randn(2, 3)

# 使用 view 函数将其形状调整为 (3, 2)
reshaped_tensor = tensor.view(tensor.shape[0],1, -1)

print("原始张量形状:", tensor.shape)
print("调整后张量形状:", reshaped_tensor.shape)


原始张量形状: torch.Size([2, 3])
调整后张量形状: torch.Size([2, 1, 3])


In [8]:
import torch

# 创建三个形状为 (2, 3) 的张量
tensor1 = torch.randn(2, 1 ,13)
tensor2 = torch.randn(2, 1 ,13)
tensor3 = torch.randn(2, 1 ,13)

# 使用 torch.stack 将它们沿着新维度堆叠
stacked_tensor = torch.stack([tensor1, tensor2, tensor3])

print("tensor1 形状:", tensor1.shape)
print("stacked_tensor 形状:", stacked_tensor.shape)

squeeze_tensor = torch.squeeze(stacked_tensor,)
print("squeeze_tensor 形状:", squeeze_tensor.shape)


tensor1 形状: torch.Size([2, 1, 13])
stacked_tensor 形状: torch.Size([3, 2, 1, 13])
squeeze_tensor 形状: torch.Size([3, 2, 13])


In [20]:
import torch
tensor1 = torch.randn(1)  ##离散值的情况
print(tensor1)
un = torch.unsqueeze(tensor1, 0)
print(un.shape)
un = torch.unsqueeze(un, 0)
print(un.shape)
un = un.permute(2,1,0)
print(un.shape)
# 连续值的情况
tensor1 = torch.randn([1])
print(tensor1)
un = torch.unsqueeze(tensor1, 0)
print(un.shape)
un = torch.unsqueeze(un, 0)
print(un.shape)
un = un.permute(2,1,0)
print(un.shape)

tensor([-0.3066])
torch.Size([1, 1])
torch.Size([1, 1, 1])
torch.Size([1, 1, 1])
tensor([1.4721])
torch.Size([1, 1])
torch.Size([1, 1, 1])
torch.Size([1, 1, 1])


In [23]:
import torch

# 创建一个形状为 (1, 2, 3) 的张量
tensor = torch.randn(1)
print(tensor)
print("原始张量形状:", tensor.shape)

# 使用 .squeeze(0) 移除第 0 维大小为 1 的维度
squeezed_tensor = tensor.squeeze(0)

print("移除维度后张量形状:", squeezed_tensor.shape)
print(squeezed_tensor)


tensor([-2.0882])
原始张量形状: torch.Size([1])
移除维度后张量形状: torch.Size([])
tensor(-2.0882)


In [36]:
import torch

# 创建一个形状为 (4, 30) 的张量
X = torch.randn(4, 30)
#print(X)
# 提取第 22 列的所有行
agent_i_index = 2
extracted_column = X[:, 20 + agent_i_index]

print("X 的形状:", X.shape)
print("提取的列的形状:", extracted_column.shape)
print(extracted_column)


X 的形状: torch.Size([4, 30])
提取的列的形状: torch.Size([4])
tensor([0.1209, 0.1242, 0.2950, 0.1127])


In [39]:
import torch
import torch.nn as nn
import numpy as np
c = np.zeros((4, 1))
print(c.shape)
print(c)
c[0] = [1]  ####！！！ 神奇 这里将c[0]赋值为[1]，c[0]的shape还是(1,)，而不是(1,1)
print(c,c.shape)
c = torch.from_numpy(c).float()
print(c)
print(c[:,0] )

(4, 1)
[[0.]
 [0.]
 [0.]
 [0.]]
[[1.]
 [0.]
 [0.]
 [0.]] (4, 1)
tensor([[1.],
        [0.],
        [0.],
        [0.]])
tensor([1., 0., 0., 0.])


In [4]:
policy_number = 1
episode = 2
name = str(policy_number) + '_' +str(episode) 
print(f'sd{name}')

sd1_2


In [9]:
import torch

a = torch.randn(3)
b = torch.randn(3)
result = torch.mul(a, b)
print(result)
r =torch.matmul(a,b)
print(r)    

b = torch.randn(2,2, 4)
print(b)
c = torch.randn(1, 4)
print(c)
result_matrix = torch.mul(b, c)
print(result_matrix.shape)
print(result_matrix)


tensor([ 0.0198, -1.2264, -1.5745])
tensor(-2.7811)
tensor([[[-1.3292, -0.1305,  1.6532, -2.9865],
         [-1.1395, -0.5254,  0.7216,  1.1371]],

        [[ 0.0281,  0.4445, -0.6437,  0.3547],
         [-1.0617,  1.2124,  0.3283,  0.0213]]])
tensor([[ 0.7903,  0.7261, -0.4777,  1.0467]])
torch.Size([2, 2, 4])
tensor([[[-1.0504, -0.0948, -0.7897, -3.1261],
         [-0.9005, -0.3815, -0.3447,  1.1902]],

        [[ 0.0222,  0.3228,  0.3075,  0.3713],
         [-0.8390,  0.8803, -0.1568,  0.0223]]])


In [10]:
obs = {'agent_0': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'agent_1': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
print(obs.values())

dict_values([[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]])


In [17]:
import torch

# 假设 obs 和 act 是两个字典
obs = {'obs1': torch.randn( 2,3), 'obs2': torch.randn( 2,3)}
act = {'act1': torch.randn( 2,1), 'act2': torch.randn( 2,1)}
print(obs)
print(act)

# 拼接观察值和动作值
concatenated = torch.cat(list(obs.values()) + list(act.values()), 1)
print(concatenated)

print("拼接后的张量形状:", concatenated.shape)


{'obs1': tensor([[-0.9118,  0.6469,  1.2986],
        [-0.9200, -1.1836, -1.1473]]), 'obs2': tensor([[ 1.0404, -0.8691, -0.9483],
        [ 0.7368,  2.5489, -0.7975]])}
{'act1': tensor([[-0.6497],
        [ 1.2791]]), 'act2': tensor([[0.1259],
        [0.4822]])}
tensor([[-0.9118,  0.6469,  1.2986,  1.0404, -0.8691, -0.9483, -0.6497,  0.1259],
        [-0.9200, -1.1836, -1.1473,  0.7368,  2.5489, -0.7975,  1.2791,  0.4822]])
拼接后的张量形状: torch.Size([2, 8])


In [27]:
x = torch.randn(2, 39)
print(x)
agent_id_index = 0
encoder_input = torch.cat((x[:,:36] , x[:,36+agent_id_index:37+agent_id_index]), 1)

decoder_input = torch.cat((x[:,37:37+agent_id_index], x[:,37+agent_id_index+1:39]), 1)
print(encoder_input.shape)
print(decoder_input.shape)
 

tensor([[ 0.8227, -1.1238,  0.7405, -0.3586, -0.6031,  0.0795, -1.3476, -0.8057,
         -1.8946, -0.4297, -0.5290, -1.5330,  0.8859, -0.2434,  0.6919,  1.0493,
          2.2404, -0.6753, -0.5856,  0.7311, -0.5163, -0.2467, -1.4401,  0.0722,
         -0.5268, -0.7519, -1.6446, -0.8844, -1.2657,  0.1851, -0.7639,  0.0550,
          0.8339, -1.4746, -1.2253, -1.2504,  0.3151, -0.0917,  2.2522],
        [ 0.6784,  0.5855, -0.1163,  0.7948,  0.4949, -0.3282, -0.5958, -0.2177,
         -0.8419,  0.1144, -1.7315, -0.2941,  1.0786, -0.0516, -2.8019, -0.5588,
         -0.7500, -0.4759,  0.0139, -0.6794,  0.3855,  0.6779, -0.1982,  0.4127,
          1.5652, -0.2581, -0.0434, -2.3590, -1.5473, -0.6892, -0.6620, -0.0309,
          2.0777,  1.2134,  2.0234,  0.2865, -0.7880, -1.8601, -1.1366]])
torch.Size([2, 37])
torch.Size([2, 1])


In [ ]:
for i in range(3):
    print(i)

In [5]:
from smacv2.env.starcraft2.maps import smac_maps
from pysc2 import maps as pysc2_maps
from smacv2.env.pettingzoo import StarCraft2PZEnv as sc2
import pytest
from pettingzoo import test
import pickle

smac_map_registry = smac_maps.get_smac_map_registry()
all_maps = pysc2_maps.get_maps()
map_names = []
for map_name in smac_map_registry.keys():
    map_class = all_maps[map_name]
    if map_class.path:
        map_names.append(map_name)


@pytest.mark.parametrize(("map_name"), map_names)
def test_env(map_name):
    env = sc2.env(map_name=map_name)
    test.api_test(env)
    # test.parallel_api_test(sc2_v0.parallel_env()) # does not pass it due to
    # illegal actions test.seed_test(sc2.env, 50) # not required, sc2 env only
    # allows reseeding at initialization
    test.render_test(env)

    recreated_env = pickle.loads(pickle.dumps(env))
    test.api_test(recreated_env)

test_env('3m')

UnboundLocalError: cannot access local variable 'agent_type' where it is not associated with a value

In [12]:
import os
import sys
import inspect
from pettingzoo import test
from smacv2.env.pettingzoo import StarCraft2PZEnv as sc2
import pickle

current_dir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe()))
)
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)


if __name__ == "__main__":
    env = sc2.env(map_name="corridor")
    test.api_test(env)
    # test.parallel_api_test(sc2_v0.parallel_env()) # does not pass it due to
    # illegal actions test.seed_test(sc2_v0.env, 50) # not required, sc2 env
    # only allows reseeding at initialization

    recreated_env = pickle.loads(pickle.dumps(env))
    test.api_test(recreated_env)

Starting API test


d:\softwares\anaconda3\envs\myrl\Lib\site-packages\pettingzoo\utils\conversions.py:252: UserWarning: The base environment `sc2` does not have a `render_mode` defined.
  warnings.warn(


TypeError: smac_parallel_env.reset() got an unexpected keyword argument 'seed'

In [8]:
import pytest
import pickle
from smacv2.env.starcraft2.maps import smac_maps
from pysc2 import maps as pysc2_maps
from smacv2.env.pettingzoo import StarCraft2PZEnv as sc2
from pettingzoo import test
from pettingzoo.utils import wrappers

def create_env(map_name, seed=None):
    env = sc2.env(map_name=map_name)
    if seed is not None:
        env.seed(seed)
    return env

def test_env(map_name):
    env = create_env(map_name, seed=50)
    env = wrappers.TerminateIllegalWrapper(env, illegal_reward=-1)
    env = wrappers.AssertOutOfBoundsWrapper(env)

    test.api_test(env)

    recreated_env = pickle.loads(pickle.dumps(env))
    test.api_test(recreated_env)

if __name__ == "__main__":
    try:
        env = create_env("corridor", seed=50)
        test.api_test(env)
        recreated_env = pickle.loads(pickle.dumps(env))
        test.api_test(recreated_env)
    except Exception as e:
        print(f"An error occurred: {e}")


An error occurred: 'parallel_to_aec_wrapper' object has no attribute 'seed'


In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import random
import numpy as np
from smacv2.env.pettingzoo import StarCraft2PZEnv


def main():
    """
    Runs an env object with random actions.
    """
    env = StarCraft2PZEnv.env()
    episodes = 10

    total_reward = 0
    done = False
    completed_episodes = 0

    while completed_episodes < episodes:
        env.reset()
        for agent in env.agent_iter():
            env.render()

            obs, reward, done, _ = env.last()
            total_reward += reward
            if done:
                action = None
            elif isinstance(obs, dict) and "action_mask" in obs:
                action = random.choice(np.flatnonzero(obs["action_mask"]))
            else:
                action = env.action_spaces[agent].sample()
            env.step(action)

        completed_episodes += 1

    env.close()

    print("Average total reward", total_reward / episodes)


if __name__ == "__main__":
    main()

UnboundLocalError: cannot access local variable 'agent_type' where it is not associated with a value

In [3]:
import numpy as np

# 智能体位置
agent_pos = np.array([1.0, 2.0])

# 敌人位置
enemy_pos = np.array([4.0, 6.0])

# 计算欧几里得距离
distance = np.linalg.norm(agent_pos - enemy_pos)

print("欧几里得距离:", distance)


TypeError: unsupported operand type(s) for -: 'list' and 'list'